In [341]:
# Read and Import
import os
#os.chdir('titanic')

import pandas as pd

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [342]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [343]:
# Features and Target
# train_f=train.drop(['Survived'],axis=1)
# train_t=train['Survived']
train_f=train

In [344]:
# Missing Values
for i in train_f.columns:
    print(i,train_f[i].isnull().sum())

PassengerId 0
Survived 0
Pclass 0
Name 0
Sex 0
Age 177
SibSp 0
Parch 0
Ticket 0
Fare 0
Cabin 687
Embarked 2


In [345]:
# New column - Salutation, to impute Age
import re

train_f['Salutation']=train_f['Name'].apply(lambda x : list(re.findall(' ([A-Za-z]{1,4})\.',x))[0] if list(re.findall(' ([A-Za-z]{1,4})\.',x)) else 'None')


In [346]:
# Find Median Age to impute
age_med=train_f.groupby('Salutation').agg({'Age':'median'}).reset_index().rename(columns={'Age':'Med_Age'})
age_med

,Salutation,Med_Age
0,Capt,70.0
1,Col,58.0
2,Don,40.0
3,Dr,46.5
4,Lady,48.0
5,Miss,21.0
6,Mlle,24.0
7,Mme,24.0
8,Mr,30.0
9,Mrs,35.0


In [347]:
# Impute missing ages
train_f=pd.merge(train_f,age_med,on='Salutation',how='left')
train_f['Age'].fillna(train_f['Med_Age'],inplace=True)
train_f

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Salutation,Med_Age
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,30.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,35.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,21.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,35.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Rev,46.5
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss,21.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.0,1,2,W./C. 6607,23.4500,NaN,S,Miss,21.0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr,30.0


In [348]:
# def isNaN(num):
#     return num != num

# def enum(l):
#     res=''
#     for i in set(l):
#         res=res+str(l.count(i))+str(i)
#     return res


train_f['cabin_cat']=train_f['Cabin'].apply(lambda x: str(x).strip()[0])
#train_f['cabin_cat']=train_f['cabin_cat'].replace('n',np.NaN)
train_f

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Salutation,Med_Age,cabin_cat
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,30.0,n
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,35.0,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,21.0,n
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,35.0,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,30.0,n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Rev,46.5,n
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss,21.0,B
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.0,1,2,W./C. 6607,23.4500,NaN,S,Miss,21.0,n
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr,30.0,C


In [349]:
# Analyze Fare to backfill cabin
import numpy as np

fare_analysis= pd.pivot_table(train_f,index=['Fare'],columns=['cabin_cat'],values='PassengerId',aggfunc=len).reset_index()
fare_analysis.fillna(-99,inplace=True)
fare_analysis.sort_values(by='Fare',ascending=False,inplace=True)
print(fare_analysis.to_string())



cabin_cat      Fare     A     B     C     D     E     F     G     T     n
247        512.3292 -99.0   2.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0   1.0
246        263.0000 -99.0 -99.0   4.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0
245        262.3750 -99.0   2.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0
244        247.5208 -99.0   2.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0
243        227.5250 -99.0 -99.0   2.0 -99.0 -99.0 -99.0 -99.0 -99.0   2.0
242        221.7792 -99.0 -99.0   1.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0
241        211.5000 -99.0 -99.0   1.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0
240        211.3375 -99.0   3.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0
239        164.8667 -99.0 -99.0   1.0 -99.0 -99.0 -99.0 -99.0 -99.0   1.0
238        153.4625 -99.0 -99.0   3.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0
237        151.5500 -99.0 -99.0   3.0 -99.0 -99.0 -99.0 -99.0 -99.0   1.0
236        146.5208 -99.0   2.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0 -99.0
235        135.6333 -99.0 -99.0   2.0 

In [350]:
# Get the majority cabin to be assigned as the cabin for that fare

tmp_cab=[]
for i,j in fare_analysis.sort_values(by='Fare',ascending=False).iterrows():
    cols='ABCDEFGT'
    tmp=list(j[list(cols)])
    if max(tmp)==-99:
        tmp_cab.append([j['Fare'],tmp_cab[-1][1]])
    else:
        ind_max=tmp.index(max(tmp))
        tmp_cab.append([j['Fare'],cols[ind_max]])

# Based on predicted cabin(tmp_cab) impute missing cabins

cab_impute=pd.DataFrame(tmp_cab,columns=['Fare','impute_cab'])
#print(cab_impute.to_string())

# Join it back with main dataframe to fill the missing cabin values and impute

train_f_merge=pd.merge(train_f,cab_impute,on='Fare',how='left')
train_f_merge['cabin_cat'][train_f_merge['cabin_cat']=='n']=train_f_merge['impute_cab']


train_f_merge

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Salutation,Med_Age,cabin_cat,impute_cab
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,30.0,F,F
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,35.0,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,21.0,E,E
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,35.0,C,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,30.0,E,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Rev,46.5,D,D
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss,21.0,B,D
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.0,1,2,W./C. 6607,23.4500,NaN,S,Miss,21.0,E,E
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr,30.0,C,D


In [351]:
# Extract features
cols=['Sex','Age','SibSp','Parch','Fare','Embarked','cabin_cat','Pclass','Survived']
train_filt=train_f_merge[cols]
train_filt.dtypes

#One hot encoding for categorical variables
train_feat_onehot=pd.get_dummies(train_filt)

# Build Features and Target
train_features=train_feat_onehot.drop(['Survived'],axis=1)
train_target=train_feat_onehot['Survived']


# Scaling Continuous values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Scale and Encode Separate Columns
train_features['Age']  = scaler.fit_transform(train_features['Age'].values.reshape(-1,1)) 
train_features['Fare']  = scaler.fit_transform(train_features['Fare'].values.reshape(-1,1)) 
train_features

,Age,SibSp,Parch,Fare,Pclass,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,cabin_cat_A,cabin_cat_B,cabin_cat_C,cabin_cat_D,cabin_cat_E,cabin_cat_F,cabin_cat_G,cabin_cat_T
0,-0.557772,1,0,-0.502445,3,0,1,0,0,1,0,0,0,0,0,1,0,0
1,0.649453,1,0,0.786845,1,1,0,1,0,0,0,0,1,0,0,0,0,0
2,-0.255965,0,0,-0.488854,3,1,0,0,0,1,0,0,0,0,1,0,0,0
3,0.423099,1,0,0.420730,1,1,0,0,0,1,0,0,1,0,0,0,0,0
4,0.423099,0,0,-0.486337,3,0,1,0,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-0.180514,0,0,-0.386671,2,0,1,0,0,1,0,0,0,1,0,0,0,0
887,-0.784126,0,0,-0.044381,1,1,0,0,0,1,0,1,0,0,0,0,0,0
888,-0.633223,1,2,-0.176263,3,1,0,0,0,1,0,0,0,0,1,0,0,0
889,-0.255965,0,0,-0.044381,1,0,1,1,0,0,0,0,1,0,0,0,0,0


In [352]:
# follow the same pre-processing steps for test

for i in test_merge.columns:
    print(i,test_merge[i].isnull().sum())

PassengerId 0
Pclass 0
Name 0
Sex 0
Age 0
SibSp 0
Parch 0
Ticket 0
Fare 0
Cabin 327
Embarked 0
Salutation 0
Med_Age 1
cabin_cat 0
impute_cab 46
impute_cab2 373


In [353]:
# Impute Age based on Salutation

import re
test['Salutation']=test['Name'].apply(lambda x : list(re.findall(' ([A-Za-z]{1,4})\.',x))[0] if list(re.findall(' ([A-Za-z]{1,4})\.',x)) else 'None')

# Merge Salutation of train to test to get the age

test=pd.merge(test,age_med,on='Salutation',how='left')
test['Age'].fillna(test['Med_Age'],inplace=True)
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Salutation,Med_Age
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr,30.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Mrs,35.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Mr,30.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Mr,30.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Mrs,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,30.0,0,0,A.5. 3236,8.0500,NaN,S,Mr,30.0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,Dona,NaN
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr,30.0
416,1308,3,"Ware, Mr. Frederick",male,30.0,0,0,359309,8.0500,NaN,S,Mr,30.0


In [354]:
#1.Impute Cabin

#1.1 Extract cabin category
test['cabin_cat']=test['Cabin'].apply(lambda x: str(x).strip()[0])
test


#1.2 Analyze Fare to backfill cabin

#1.2.1 Using train backfill data to fill in


test_merge=pd.merge(test,cab_impute,on='Fare',how='left')
test_merge['cabin_cat'][test_merge['cabin_cat']=='n']=test_merge['impute_cab']
test_merge

#1.2.2 Use test backfill data to fill in cabin_cat values that are still missing

import numpy as np

fare_analysis_test= pd.pivot_table(test,index=['Fare'],columns=['cabin_cat'],values='PassengerId',aggfunc=len).reset_index()
fare_analysis_test.fillna(-99,inplace=True)
fare_analysis_test.sort_values(by='Fare',ascending=False,inplace=True)

# Filter fares that are not in training fare analysis dataset
fare_analysis_test=fare_analysis_test[~fare_analysis_test['Fare'].isin(fare_analysis['Fare'])]
fare_analysis_test

# 1.2.3 Get the majority cabin to be assigned as the cabin for that fare

tmp_cab=[]
for i,j in fare_analysis_test.sort_values(by='Fare',ascending=False).iterrows():
    cols='ABCDEFG'
    tmp=list(j[list(cols)])
    if max(tmp)==-99:
        tmp_cab.append([j['Fare'],tmp_cab[-1][1]])
    else:
        ind_max=tmp.index(max(tmp))
        tmp_cab.append([j['Fare'],cols[ind_max]])

# Based on predicted cabin(tmp_cab) impute missing cabins
cab_impute_test=pd.DataFrame(tmp_cab,columns=['Fare','impute_cab2'])


# 1.2.4 Impute missing cabins the to completely fill all missing values of cabin

test_merge=pd.merge(test_merge,cab_impute_test,on='Fare',how='left')#['cabin_cat'].isnull().sum()

test_merge['cabin_cat']=test_merge['cabin_cat'].fillna(test_merge['impute_cab2'])

test_merge['cabin_cat']=test_merge['cabin_cat'].fillna('F') # fill the last remaining cabin with F
test_merge['Fare']=test_merge['Fare'].fillna(0) # fill missing fare as Zero

test_merge['cabin_cat'].unique()






/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


array(['E', 'C', 'G', 'F', 'B', 'A', 'D'], dtype=object)

In [363]:
# Extract Features and one hot encoding
cols=['Sex','Age','SibSp','Parch','Fare','Embarked','cabin_cat','Pclass']
test_feat_onehot=pd.get_dummies(test_merge[cols])
test_feat_onehot['cabin_cat_T']=0
test_features=test_feat_onehot

# Scaling Continuous values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Scale and Encode Separate Columns
test_features['Age']  = scaler.fit_transform(test_features['Age'].values.reshape(-1,1)) 
test_features['Fare']  = scaler.fit_transform(test_features['Fare'].values.reshape(-1,1)) 
test_features

print(test_features.columns)
print(train_features.columns)

Index(['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S', 'cabin_cat_A', 'cabin_cat_B',
       'cabin_cat_C', 'cabin_cat_D', 'cabin_cat_E', 'cabin_cat_F',
       'cabin_cat_G', 'cabin_cat_T'],
      dtype='object')
Index(['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S', 'cabin_cat_A', 'cabin_cat_B',
       'cabin_cat_C', 'cabin_cat_D', 'cabin_cat_E', 'cabin_cat_F',
       'cabin_cat_G', 'cabin_cat_T'],
      dtype='object')


In [407]:
# Begin Modelling

#1. Test Train Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_features, train_target, test_size=0.33, random_state=42)


# 2. Train the model using Gradient Boosting

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

model = DecisionTreeClassifier(criterion='entropy', max_depth=1)
AdaBoost = AdaBoostClassifier(base_estimator=model, n_estimators=5000, learning_rate=0.1)

# 2.1 Fit the model with training data
boostmodel = AdaBoost.fit(X_train, y_train)

# Get score on test data
y_pred = boostmodel.predict(X_test)
predictions = metrics.accuracy_score(y_test, y_pred)
print('The accuracy is: ', predictions * 100, '%')

The accuracy is:  80.0 %


In [396]:
# Make Final Prediction on test data

final_pred_ada = boostmodel.predict(test_features)
final_pred_ada

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [387]:
from datetime import datetime
dt_tm_stmp=str(datetime.now().strftime("%Y-%m-%d %H%M"))



titanic_submission_ada=test_merge[['PassengerId','Age']]
titanic_submission_ada['Survived']=pd.Series(final_pred_ada)
titanic_submission_ada.drop(['Age'],axis=1,inplace=True)
titanic_submission_ada.to_csv('titanic_submission_ada_{0}.csv'.format(dt_tm_stmp),index=False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [417]:
# Using Gradient boost

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgbmodel = XGBClassifier()
xgbmodel.fit(X_train, y_train)

#Model Predictions on test data

y_pred = xgbmodel.predict(X_test)
predictions = [round(value) for value in y_pred]

# Print Accruact

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


# Make Final Prediction on test data

final_pred_xgb = xgbmodel.predict(test_features)
final_pred_xgb

Accuracy: 80.34%


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [399]:
from datetime import datetime
dt_tm_stmp=str(datetime.now().strftime("%Y-%m-%d %H%M"))



titanic_submission_xgb=test_merge[['PassengerId','Age']]
titanic_submission_xgb['Survived']=pd.Series(final_pred_xgb)
titanic_submission_xgb.drop(['Age'],axis=1,inplace=True)
titanic_submission_xgb.to_csv('titanic_submission_xgb_{0}.csv'.format(dt_tm_stmp),index=False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [419]:
X_test.shape

(295, 18)

In [460]:
# Neural Network

# neural network with keras 
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

# define the keras models
from keras.layers import LeakyReLU

for i in ['']:
    exec('model_{0} = Sequential()'.format(i))
    exec('model_{0}.add(Dense(20, input_dim=18))'.format(i))
    exec('model_{0}.add(LeakyReLU(alpha=0.02))'.format(i))
    exec('model_{0}.add(Dense(20))'.format(i))
    exec('model_{0}.add(LeakyReLU(alpha=0.02))'.format(i))
    exec('model_{0}.add(Dense(20))'.format(i))
    exec('model_{0}.add(LeakyReLU(alpha=0.02))'.format(i))
    exec('model_{0}.add(Dense(8))'.format(i))
    exec('model_{0}.add(LeakyReLU(alpha=0.02))'.format(i))
    exec('model_{0}.add(Dense(8))'.format(i))
    exec('model_{0}.add(LeakyReLU(alpha=0.02))'.format(i))
    exec('model_{0}.add(Dense(1))'.format(i))
    exec('model_{0}.add(LeakyReLU(alpha=0.02))'.format(i))

In [461]:
# compile the keras model

model_.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# fit the model

model_.fit(X_train, y_train, epochs=50, batch_size=10)

Epoch 1/50
60/60 [==============================] - 0s 1ms/step - loss: 0.2296 - accuracy: 0.6493
Epoch 2/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1821 - accuracy: 0.7416
Epoch 3/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1611 - accuracy: 0.7785
Epoch 4/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1510 - accuracy: 0.7852
Epoch 5/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1466 - accuracy: 0.7987
Epoch 6/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1423 - accuracy: 0.7987
Epoch 7/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1353 - accuracy: 0.8104
Epoch 8/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1331 - accuracy: 0.8104
Epoch 9/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1302 - accuracy: 0.8272
Epoch 10/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1265 - accuracy: 0.8221
Epoch 11/50
60/60 [

In [463]:
# Evaluate the model

_, accuracy = model_.evaluate(X_test, y_test)
final_pred_nn=[round(i[0]) for i in final_pred_nn]
print('Accuracy for dir model: %.2f' % (accuracy*100))

10/10 [==============================] - 0s 1ms/step - loss: 0.1528 - accuracy: 0.8136
Accuracy for dir model: 81.36


In [472]:
# Final Prediction

final_pred_nn=model_.predict(test_features)
final_pred_nn=[round(i[0]) for i in final_pred_nn]

In [473]:
from datetime import datetime
dt_tm_stmp=str(datetime.now().strftime("%Y-%m-%d %H%M"))



titanic_submission_nn=test_merge[['PassengerId','Age']]
titanic_submission_nn['Survived']=pd.Series(final_pred_nn)
titanic_submission_nn.drop(['Age'],axis=1,inplace=True)
titanic_submission_nn.to_csv('titanic_submission_nn_{0}.csv'.format(dt_tm_stmp),index=False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [474]:
sum(final_pred_nn)
# This is an untracked change for github

150